In [1]:
import sys
import os
sys.path.append('/share/software/user/open/py-matplotlib/3.7.1_py39/lib/python3.9/site-packages')
sys.path.append('/share/software/user/open/py-numpy/1.24.2_py39/lib/python3.9/site-packages')
sys.path.append('/share/software/user/open/py-pandas/2.0.1_py39/lib/python3.9/site-packages')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

In [2]:
cache_path = '/scratch/groups/bhgood/uhgg/pairwise_stats/LE/syn/MGYG-HGUT-02492/full_cache'
stat_path = '/scratch/groups/bhgood/uhgg/pairwise_stats/LE/syn/MGYG-HGUT-02492/'

In [5]:
%%time
# first bootstrapping the frequency scan

B = 1000 # num of bootstrap samples
num_ells = 3
num_f0s = len(np.load(os.path.join(stat_path, 'f_scan_f0s_{}.npy'.format(0))))
bootstrap_res = np.zeros((num_ells, num_f0s, B))

for ell_idx in range(num_ells):
    f0s = np.load(os.path.join(stat_path, 'f_scan_f0s_{}.npy'.format(ell_idx)))
    num_f0s = len(f0s)
    for f_idx in range(num_f0s):
        numers = np.load(os.path.join(cache_path, 'f_scan_stats_numer_full_{}_{}.npy'.format(ell_idx, f_idx)))
        denoms = np.load(os.path.join(cache_path, 'f_scan_stats_denom_full_{}_{}.npy'.format(ell_idx, f_idx)))
        for b in range(B):
            sampled_idxs = np.random.choice(np.arange(len(numers)), len(numers))
            sampled_numers = numers[sampled_idxs]
            sampled_denoms = denoms[sampled_idxs]
            sampled_LE = sampled_numers.mean() / sampled_denoms.mean()
            bootstrap_res[ell_idx, f_idx, b] = sampled_LE

CPU times: user 1h 3min 46s, sys: 1min 30s, total: 1h 5min 17s
Wall time: 1h 5min 27s


In [6]:
# saving the bootstrap res
np.save('fig10b_bootstrap', bootstrap_res)

In [38]:
# computing the confidence intervals

bootstrap_res = np.load('fig10b_bootstrap.npy')
bootstrap_mean = bootstrap_res.mean(axis=2)
# calculate CI on log scale
log_bootstrap = np.log10(bootstrap_res)

confidence_level = 0.99
lower_percentile = (1.0 - confidence_level) / 2.0 * 100
upper_percentile = (1.0 + confidence_level) / 2.0 * 100

confidence_interval = np.percentile(bootstrap_res, [lower_percentile, upper_percentile], axis=2)
ci_lower = confidence_interval[0]
ci_upper = confidence_interval[1]

np.save('fig10b_ci', confidence_interval)

In [4]:
%%time
# then bootstrapping the ell scan

B = 1000 # num of bootstrap samples
num_f0s = 6
num_ells = len(np.load(os.path.join(stat_path, 'ell_scan_ells_{}.npy'.format(0))))
bootstrap_res = np.zeros((num_f0s, num_ells, B))
    
for f_idx in range(num_f0s):
    ells = np.load(os.path.join(stat_path, 'ell_scan_ells_{}.npy'.format(f_idx)))
    for ell_idx in range(num_ells):
        if ell_idx%20==0:
            print(f_idx, ell_idx)
        numers = np.load(os.path.join(cache_path, 'ell_scan_stats_numer_full_{}_{}.npy'.format(f_idx, ell_idx)))
        denoms = np.load(os.path.join(cache_path, 'ell_scan_stats_denom_full_{}_{}.npy'.format(f_idx, ell_idx)))
        for b in range(B):
            sampled_idxs = np.random.choice(np.arange(len(numers)), len(numers))
            sampled_numers = numers[sampled_idxs]
            sampled_denoms = denoms[sampled_idxs]
            sampled_LE = sampled_numers.mean() / sampled_denoms.mean()
            bootstrap_res[f_idx, ell_idx, b] = sampled_LE

0 0
0 20
0 40
0 60
0 80
0 100
0 120
0 140
0 160
0 180
0 200
1 0
1 20
1 40
1 60
1 80
1 100
1 120
1 140
1 160
1 180
1 200
2 0
2 20
2 40
2 60
2 80
2 100
2 120
2 140
2 160
2 180
2 200
3 0
3 20
3 40
3 60
3 80
3 100
3 120
3 140
3 160
3 180
3 200
4 0
4 20
4 40
4 60
4 80
4 100
4 120
4 140
4 160
4 180
4 200
5 0
5 20
5 40
5 60
5 80
5 100
5 120
5 140
5 160
5 180
5 200
CPU times: user 4min 59s, sys: 17.2 s, total: 5min 16s
Wall time: 5min 34s


In [ ]:
np.save('fig10a_bootstrap', bootstrap_res)

In [36]:
# computing the confidence intervals

bootstrap_res = np.load('fig10a_bootstrap.npy')
bootstrap_mean = bootstrap_res.mean(axis=2)
# calculate CI on log scale
log_bootstrap = np.log10(bootstrap_res)

confidence_level = 0.99
lower_percentile = (1.0 - confidence_level) / 2.0 * 100
upper_percentile = (1.0 + confidence_level) / 2.0 * 100

confidence_interval = np.percentile(bootstrap_res, [lower_percentile, upper_percentile], axis=2)
ci_lower = confidence_interval[0]
ci_upper = confidence_interval[1]

np.save('fig10a_ci', confidence_interval)